In [1]:
from data import get_loader, test_dataset
from options import opt
import torch.backends.cudnn as cudnn
import logging
from tensorboardX import SummaryWriter
from utils import clip_gradient, adjust_lr
from models.BBSNet_model import BBSNetChannelSpatialAttention, BBSNetTransformerAttention
from torchvision.utils import make_grid
from datetime import datetime
import numpy as np
import os
import torch
import torch.nn.functional as F
import sys
sys.path.append('./models')

from BBSNet_train import *
from BBSNet_test import *
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0
load data...
Start train...
2025-08-15 15:14:22.729093 Epoch [001/002], Step [0001/2185], Loss1: 0.7465 Loss2: 0.6646
2025-08-15 15:16:36.438211 Epoch [001/002], Step [0100/2185], Loss1: 0.6136 Loss2: 0.5898
Keyboard Interrupt: save model and exit.
save checkpoints successfully!


KeyboardInterrupt: 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
# test(method='BBSNetChannelSpatialAttention')

In [ ]:
# evaluate()

In [ ]:
draw_curve()